In [1]:
%pip install semantic-link --quiet 
import synapse.ml.core
import pandas as pd
from synapse.ml.services import *
from pyspark.sql import SQLContext
from pyspark.sql.functions import col, flatten, udf, lower, trim
from pyspark.sql.types import StringType
import sempy
import sempy.fabric as fabric
import base64
import json
import copy

StatementMeta(, d255bb4f-52db-4b8d-87d1-07c10b1a1b9a, 8, Finished, Available)


[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.



# Parameters & initialize

In [2]:
# Parameters
workspaceId = fabric.get_workspace_id()
reportId = "1400d874-2759-457c-aacf-e5de10ba80cc"

languages = ["fr", "it", "hi", "tlh-Latn", "ja", "hu" ]
#Klingon-tlh-Latn;Japanese-ja; Hindi-hi; Hungarian-hu;Maori-mi;Thai-th

newReportName = "[trans] - Sales"

# Client initialization
client = fabric.FabricRestClient()

StatementMeta(, d255bb4f-52db-4b8d-87d1-07c10b1a1b9a, 10, Finished, Available)

# Cleanup for the demo

In [45]:
workspaceId = fabric.get_workspace_id()
response = client.get(f"/v1/workspaces/{workspaceId}/items")

listItems = response.json()['value']

itemsToDelete = [x for x in listItems if x['type'] == 'Report' and x['displayName'].startswith(newReportName)]

for item in itemsToDelete:
    itemId = item['id']
    
    print(f"Deleting report '{itemId}'")

    client.delete(f"/v1/workspaces/{workspaceId}/items/{itemId}")

StatementMeta(, 33a0396e-97e8-4718-ade4-9b5f87abc375, 65, Finished, Available)

Deleting report 'cbc60c23-f63c-4dba-b217-eee4239ca649'
Deleting report 'c6b92495-4d8a-4deb-805d-2d0e70d075a4'
Deleting report '03c49cb7-750b-413d-87da-a7d7b15fb03a'
Deleting report '2220a978-1204-4c81-89c8-ac3abbcc9143'
Deleting report 'a72a4538-51f8-43b5-82ce-9255c31b3050'
Deleting report '426da9d9-76cd-4026-9a8b-64bbefafbbff'
Deleting report 'df09ebd5-c84b-4eb6-b218-e80cf10550ba'


# Get report definition parts

In [3]:
response = client.post(f"/v1/workspaces/{workspaceId}/items/{reportId}/getDefinition")

jsonParts = response.json()['definition']['parts']

df_parts = pd.json_normalize(jsonParts)
pd.set_option('display.max_colwidth', 100) 
df_parts

StatementMeta(, d255bb4f-52db-4b8d-87d1-07c10b1a1b9a, 11, Finished, Available)

,path,payload,payloadType
0,definition.pbir,ewogICJ2ZXJzaW9uIjogIjQuMCIsCiAgImRhdGFzZXRSZWZlcmVuY2UiOiB7CiAgICAiYnlQYXRoIjogbnVsbCwKICAgICJi...,InlineBase64
1,StaticResources/SharedResources/BaseThemes/CY23SU04.json,ewogICJuYW1lIjogIkNZMjNTVTA0IiwKICAiZGF0YUNvbG9ycyI6IFsKICAgICIjMTE4REZGIiwKICAgICIjMTIyMzlFIiwK...,InlineBase64
2,StaticResources/RegisteredResources/Light4437032645752863.json,ewogICIkc2NoZW1hIjogImh0dHBzOi8vcmF3LmdpdGh1YnVzZXJjb250ZW50LmNvbS9taWNyb3NvZnQvcG93ZXJiaS1kZXNr...,InlineBase64
3,StaticResources/RegisteredResources/_7abfc6c7-1a23-4b5f-bd8b-8dc472366284171093267.jpg,77+9UE5HDQoaCgAAAA1JSERSAAACAAAAAgAIBgAAAO+/vXjvv73vv70AAAAJcEhZcwAACxMAAAsTAQDvv73vv70YAAAAAXNS...,InlineBase64
4,definition/version.json,ew0KICAiJHNjaGVtYSI6ICJodHRwczovL2RldmVsb3Blci5taWNyb3NvZnQuY29tL2pzb24tc2NoZW1hcy9mYWJyaWMvaXRl...,InlineBase64
5,definition/report.json,ew0KICAiJHNjaGVtYSI6ICJodHRwczovL2RldmVsb3Blci5taWNyb3NvZnQuY29tL2pzb24tc2NoZW1hcy9mYWJyaWMvaXRl...,InlineBase64
6,definition/pages/pages.json,ew0KICAiJHNjaGVtYSI6ICJodHRwczovL2RldmVsb3Blci5taWNyb3NvZnQuY29tL2pzb24tc2NoZW1hcy9mYWJyaWMvaXRl...,InlineBase64
7,definition/pages/ReportSection89a9619c7025093ade1c/page.json,ew0KICAiJHNjaGVtYSI6ICJodHRwczovL2RldmVsb3Blci5taWNyb3NvZnQuY29tL2pzb24tc2NoZW1hcy9mYWJyaWMvaXRl...,InlineBase64
8,definition/pages/ReportSection89a9619c7025093ade1c/visuals/0f9bcdc145fc9e6fdc33/visual.json,ew0KICAiJHNjaGVtYSI6ICJodHRwczovL2RldmVsb3Blci5taWNyb3NvZnQuY29tL2pzb24tc2NoZW1hcy9mYWJyaWMvaXRl...,InlineBase64
9,definition/pages/ReportSection89a9619c7025093ade1c/visuals/223d9300700274da67c8/visual.json,ew0KICAiJHNjaGVtYSI6ICJodHRwczovL2RldmVsb3Blci5taWNyb3NvZnQuY29tL2pzb24tc2NoZW1hcy9mYWJyaWMvaXRl...,InlineBase64


# Enumerate all the report visuals **with** titles

In [4]:
visualFiles = [part for part in jsonParts if part["path"].endswith('visual.json')]

visualsDF = pd.DataFrame({'path': [], 'visualID': [], 'title': []})

for visualFile in visualFiles:

    visualContent = base64.b64decode(visualFile['payload']).decode('utf-8')

    visualContentJson = json.loads(visualContent)

    visualName = visualContentJson['name']
    
    try:
        visualTitle = visualContentJson["visual"]["visualContainerObjects"]["title"][0]["properties"]["text"]["expr"]["Literal"]["Value"]
    except:
        visualTitle = ""

    if visualTitle != "":
        visualTitle = visualTitle.strip("'")
        visualsDF = pd.concat([visualsDF, pd.DataFrame({'path': visualFile['path'], 'visualID': visualName, 'title': visualTitle}, index=[0])], ignore_index=True)

visualsDF

StatementMeta(, d255bb4f-52db-4b8d-87d1-07c10b1a1b9a, 12, Finished, Available)

,path,visualID,title
0,definition/pages/ReportSection89a9619c7025093ade1c/visuals/32b08d7f32cb5f9816cc/visual.json,32b08d7f32cb5f9816cc,Sales Amount by Brand and Gender
1,definition/pages/ReportSection89a9619c7025093ade1c/visuals/484fbdd73143c5bf71fa/visual.json,484fbdd73143c5bf71fa,"Sales Amount, Sales Amount (LY), Margin and Sales Amount (12M average) by Date (Year-Month)"
2,definition/pages/ReportSection89a9619c7025093ade1c/visuals/4b18758374093c2e23fb/visual.json,4b18758374093c2e23fb,Sales Amount vs # Customers
3,definition/pages/ReportSection89a9619c7025093ade1c/visuals/5acb1caf298449a8acb4/visual.json,5acb1caf298449a8acb4,Sales Amount and Sales Amount (LY) by Category
4,definition/pages/ReportSectiond90903559771e58905a1/visuals/a64038428c095bd71739/visual.json,a64038428c095bd71739,Sales Amount by Country and Gender
5,definition/pages/ReportSection8a2d1e93c8442763827c/visuals/6f9803040a3195450ea1/visual.json,6f9803040a3195450ea1,Value by Product
6,definition/pages/ReportSection8a2d1e93c8442763827c/visuals/e50725a866c432900b35/visual.json,e50725a866c432900b35,Value by Date
7,definition/pages/ReportSection8a2d1e93c8442763827c/visuals/e859ee4d7c5b10e836d6/visual.json,e859ee4d7c5b10e836d6,"Value, Value (ly) and Value Daily Max by Month Start Date"
8,definition/pages/ReportSection8a2d1e93c8442763827c/visuals/f0143b84500069035233/visual.json,f0143b84500069035233,Value (ytd) by Month Start Date


# Call Azure AI translator 

In [5]:
# Initialize Azure AI translator

translate = (Translate()
    .setTextCol("title")
    .setToLanguage(languages)
    .setOutputCol("translation")
    .setConcurrency(5))

# Call Azure AI translator to translate all the visual titles into multiple languages

visualsDF_spark = sqlContext.createDataFrame(visualsDF)

translatedDF = translate.transform(visualsDF_spark)\
        .withColumn("translation_result", flatten(col("translation.translations")))\
        .withColumn("translation", col("translation_result.text")[0])\
        .cache()

display(translatedDF)

translatedDF = translatedDF.toPandas()

StatementMeta(, d255bb4f-52db-4b8d-87d1-07c10b1a1b9a, 13, Finished, Available)

SynapseWidget(Synapse.DataFrame, 89b4659e-5b5a-4e10-9843-f333a0c660cb)

/opt/spark/python/lib/pyspark.zip/pyspark/sql/pandas/conversion.py:117: UserWarning: toPandas attempted Arrow optimization because 'spark.sql.execution.arrow.pyspark.enabled' is set to true; however, failed by the reason below:
  Nested StructType not supported in conversion to Arrow
Attempting non-optimization as 'spark.sql.execution.arrow.pyspark.fallback.enabled' is set to true.


# For each language: create a report with the translated visuals

In [6]:
for lang in languages:

    translatedReportName = f"{newReportName} - {lang}"

    print(f"Creating report '{translatedReportName}'")

    for visualFile in visualFiles:
                
        # Find translation for the visual

        translatedTexts = translatedDF.loc[translatedDF['path'] == visualFile['path'], 'translation_result']

        if not translatedTexts.empty:
            
            # Get translation of the visual for the language

            translatedText = [t for t in translatedTexts.values[0] if t.to == lang][0]   

            if translatedText is not None:
                try:
                    
                    # Get visual title json property

                    visualContent = base64.b64decode(visualFile['payload']).decode('utf-8')

                    visualContentJson = json.loads(visualContent)

                    titleProperty = visualContentJson["visual"]["visualContainerObjects"]["title"][0]["properties"]["text"]["expr"]["Literal"]

                    translatedTextStr = translatedText.text
                
                    titleProperty["Value"] = f"'{translatedTextStr}'"            
                            
                    # Replace the visual part payload

                    visualFile['payload'] = base64.b64encode(json.dumps(visualContentJson).encode('utf-8')).decode('utf-8')                    
                except KeyError:
                    pass
    
    payload = {}
    payload['displayName'] = translatedReportName
    payload['type'] = "Report"
    payload['definition'] =  {
        'parts' : jsonParts
    }

    response = client.post(f"/v1/workspaces/{workspaceId}/items", json= payload)

StatementMeta(, d255bb4f-52db-4b8d-87d1-07c10b1a1b9a, 14, Submitted, Running)

Creating report '[trans] - Sales - fr'
Creating report '[trans] - Sales - it'
Creating report '[trans] - Sales - hi'
Creating report '[trans] - Sales - tlh-Latn'
Creating report '[trans] - Sales - ja'
Creating report '[trans] - Sales - hu'
